## Задание 07

### Задача 1. Смещение константного алгоритма (2 балла)
Пусть $x \in R^d$, и значение каждого признака на объекте $x$ независимо генерируется из равномерного распределения $x_i\in U[0,1],i=1,2\dots,d$. Будем считать, что объекты в выборке независимы, а $\mathbb{E}[y|x]=x^Tx.$ Найдите смещение константного алгоритма, полученного минимизацией среднеквадратичной ошибки на обучающей выборке. 

Как известно, константа наилучшим образом приближающая по среднеквадратичной ошибке это матожидание

$$c = \int_x x^Txp(x)$$


$$Q(R)(c) = \int_{X,Y} p(x, y)(y-c)^2dxdy = \int_Xp(x)(\int_Yp(y|x)(y-c)^2dy)dx =\\ \int_X (p(x)(c^2-2c\mathrm{E}(y|x)+\int y^2p(y|x)dy)dx) = c^2 - 2c\mathrm{E}(xx^T) +\mathrm{E}(xx^T)^2$$

$$\mathrm{E}(xx^T) = \frac{d}{3}$$
$$\mathrm{E}(xx^T)^2 = \frac{d}{5} + \frac{d(d-1)}{9}$$

$$bias = \frac{4d}{45}$$

### Задача 2. Разложение ошибки (2 балла)
Истинная зависимость имеет вид $y_i = 3x^2_i+ u_i$, где $y_i$ - прогнозируемая переменная, $x_i$-признак и $u_i$ - случайная составляющая. Величины $x_i$ независимы и равновероятно принимаю значения $0, 1, 2$. Величины $u_i$ независимы и равновероятно принимают значения $−1$ и $1$.
Исследователь Анатолий оценивает модель линейной регрессии $y_i = wx_i$, минимизируя среднеквадратичную ошибку.
Разложите ожидание квадрата ошибки прогноза на шум, смещение и разброс.

$$Q(w) = \int_X (3\cdot x_i^2 + u_i - w\cdot x_i)^2 \to min_w$$
$$\frac{\partial Q}{\partial w} = \int_X -(3\cdot x_i^2 + u_i - w\cdot x_i)\cdot w = 0$$
$$w = \frac{6\cdot\mathrm{E}{x_i^2} }{2\cdot\mathrm{E}x_i } = 5$$

Шум
$$\sigma^2 = \frac{1}{2}*1+\frac{1}{2}*1 = 1$$

Смещение
$$\mathrm{E}(f-\hat{f}) = \mathrm{E}(3x_i^2 - wx_i)^2 = \frac{5w^2-54w+153}{3}$$

Разброс
$$\mathrm{D}(\hat{f}) = \mathrm{D}(5x_i - wx_i)^2 = \frac{2(5-w)^2}{3} $$

### Задача 3. Взвешенное голосование (2 балла)
Рассмотрим задачу бинарной классификации, пусть у нас есть три алгоритма $b_1(x), b_2(x)$ и $b_3(x)$, каждый из которых ошибается с вероятностью $p$. Мы строим композицию взвешенным голосованием: алгоритмам присвоены значимости $w_1, w_2$ и $w_3$, и для вынесения вердикта суммируются значимости алгоритмов, проголосовавших за каждый из классов:
$$
a_0=\sum_{i=1}^3w_i[b_i(x)=0], $$
$$
a_1=\sum_{i=1}^3w_i[b_i(x)=1]. \\
$$
Объект $x$ относится к классу, для которого сумма оказалась максимальной. Например, если первые два алгоритма голосуют за класс $0$, а третий за класс $1$, то выбирается класс $0$, если $w_1+w_2>w_3$, и класс $1$ в противном случае. Какова вероятность ошибки такой композиции этих трех алгоритмов, если:

* $w_1=0.3, w_2=0.4, w_3=0.3$;
* $w_1=0.2, w_2=0.5, w_3=0.2$?


* $w_1=0.3, w_2=0.4, w_3=0.3$;

  Для того чтобы был принят верный ответ, необходимо чтобы ответило верно ответило как минимум 2 алгоритма.
  
  Тогда $\hat{p} = 3*p^2*(1-p) + p^3 = 3*p^2 - 2*p^3$
  
* $w_1=0.2, w_2=0.5, w_3=0.2$
  
  Так как один из алгоритмов имеет вес 0.5, значит решение принадлежит алгоритму 2
  
  Тогда $\hat{p} = p$

### Задача 4. Разложение ошибки с помощью бутстрапа 

В этом задании вам предстоит воспользоваться возможностями bootstraping для оценки смещения и разброса алгоритмов машинного обучения. Делать мы это будем на данных boston:

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
from sklearn.datasets import load_boston

In [4]:
boston = load_boston()
X = boston["data"]
y = boston["target"]
X.shape, y.shape

((506, 13), (506,))

#### Алгоритм оценки смещения и разброса алгоритма $a$

1. Сгенерировать $s$ выборок $X_j$ методом бутстрапа.

2. На каждой выборке $X_j$ обучить алгоритм $a_j$.

3. Для каждой выборки $X_j$ определить множество объектов $T_j$, не вошедших в нее (out-of-bag). Вычислить предсказания алгоритма $a_j$ на объектах $T_j$.

Поскольку у нас есть только один ответ для каждого объекта, мы будем считать шум равным 0, а $\mathbb{E}[y|x]$ равным имеющемуся правильному ответу для объекта $x$.

#### Итоговые оценки:

* #### Смещение: 
для одного объекта - квадрат разности среднего предсказания и правильного ответа. Среднее предсказание берется только по тем алгоритмам $a_j$, для которых этот объект входил в out-of-bag выборку $T_j$. Для получения общего смещения выполнить усреденение смещений по объектам.
* #### Разброс: 
для одного объекта - выборочная дисперсия предсказаний алгоритмов $a_j$, для которых этот объект входил в out-of-bag выборку $T_j$. Для получения общего разброса выполнить усреденение разбросов по объектам.
* #### Ошибка $L$:
усреднить квадраты разностей предсказания и правильного ответа по всем выполненным предсказаниям для всех объектов.

В результате должно получиться, что ошибка приблизительно равна сумме смещения и разброса!

* реализуйте описанный алгоритм. Обратите внимание, что если объект не вошел ни в одну из out-of-bag выборок, учитывать его в вычислении итоговых величин не нужно. (3 балла) 

* Оцените смещение, разброс и ошибку для трех алгоритмов с гиперпараметрами по умолчанию: линейная регрессия, решающее дерево, случайный лес.

* Проанализируйте полученный результат. Согласуются ли полученные результаты с теми, что мы обсуждали на занятиях? (1 балл)

In [63]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from numpy import random
from sklearn.metrics import mean_squared_error
import math

In [22]:
sample_size = y.shape[0]
s = 5

In [23]:
bootstrap = lambda : random.randint(0, sample_size, size = sample_size)
bootstrap_test = lambda x: list(set(range(sample_size)).difference(set(x)))

In [85]:
def BVD(model, s = s):
    bias = [[] for i in range(sample_size)]
    var = [[] for i in range(sample_size)]
    err = []
    
    for _ in range(s):
        md = model()
        
        ind = bootstrap()
        ind_test = bootstrap_test(ind)
        md.fit(X[ind], y[ind])
        
        for i in ind_test:
            bias[i].append((md.predict([X[i]])[0] - y[i])**2)
            var[i].append(md.predict([X[i]])[0])
        
        err.append(mean_squared_error(y[ind_test], md.predict(X[ind_test])))
        
    bias = np.mean([np.mean(b) for b in bias if len(b)>1])
    var = np.mean([np.std(b)**2 for b in var if len(b)>1])
    
    return bias, var, np.mean(err)

In [86]:
BVD(LinearRegression)

(24.205688078129953, 0.5902325369334707, 23.25246055186871)

In [87]:
BVD(RandomForestRegressor)

/Users/valerka_bvv/anaconda2/envs/py37/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/valerka_bvv/anaconda2/envs/py37/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/valerka_bvv/anaconda2/envs/py37/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/valerka_bvv/anaconda2/envs/py37/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.2

(16.874310244029072, 2.112362804863274, 16.11558692726518)

In [91]:
BVD(DecisionTreeRegressor)

(22.77690746934225, 7.878715830546266, 24.787379459738293)

Как и предполагалось, регрессия имеет большое смешение и маленький разброс, лес небольшой разброс и смешение, дерево - большой разброс